# Các thư viện cần thiết:

In [26]:
import re
import math
import datetime
import pandas as pd
from fuzzywuzzy import fuzz, process
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

# Khởi tạo Selenium:

In [2]:
# Đường dẫn đến ChromeDriver
driver_path = r'.\\chromedriver-win64\\chromedriver.exe'

# Đường dẫn đến tệp thực thi của Google Chrome
chrome_path = r'C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe'

# Cấu hình tùy chọn Chrome
options = webdriver.ChromeOptions()
options.binary_location = chrome_path

# Tạo đối tượng Service để chỉ định đường dẫn ChromeDriver
service = Service(driver_path)

# Khởi tạo ChromeDriver với Service và các tùy chọn
driver = webdriver.Chrome(service=service, options=options)

# Thời gian chờ tối đa cho một trang web được hiển thị hoàn chỉnh
wait = WebDriverWait(driver, 5)

# Tạo dataframe để lưu các việc làm thu thập được

In [3]:
# Khởi tạo DataFrame trống với các cột
columns = ['Trang thu thập', 'Tên công ty', 'Tên công việc', 'Vị trí ứng tuyển', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm',
           'Địa điểm', 'Ngày đăng tuyển', 'Lương tối thiểu', 'Lương tối đa', 'Lương TB']
df = pd.DataFrame(columns=columns)

# Thiết lập kiểu dữ liệu phù hợp
df['Ngày đăng tuyển'] = pd.to_datetime(df['Ngày đăng tuyển'], errors='coerce')
df['Lương tối thiểu'] = pd.to_numeric(df['Lương tối thiểu'], errors='coerce')
df['Lương tối đa'] = pd.to_numeric(df['Lương tối đa'], errors='coerce')
df['Lương TB'] = pd.to_numeric(df['Lương TB'], errors='coerce')

# Lọc các thông tin thu thập

## Tên công việc

In [4]:
# Từ điển từ khóa ánh xạ tên công việc
job_keywords = {
    'Lập trình viên': ['lập trình', 'developer', 'programmer', 'coder', 'software developer', 'engineer', 'dev', 
                       'backend developer', 'frontend developer', 'fullstack developer', 'software coder', 'software engineer', 
                       'python developer', 'java developer', 'c++ developer', 'mobile developer', 'golang developer', 'backend dev', 
                       'frontend dev', 'devops engineer', 'web programmer'],
    
    'Kinh doanh': ['sales', 'marketing', 'media', 'quảng cáo', 'kinh doanh', 'business', 'advertising', 'thương hiệu', 
                   'brand manager', 'digital marketing', 'sales executive', 'account manager', 'bán hàng', 'business development', 
                   'salesman', 'trưởng phòng kinh doanh', 'quản lý bán hàng', 'trade marketing', 'PR', 'quan hệ công chúng'],
    
    'Quản trị viên hệ thống': ['administrator', 'system admin', 'sysadmin', 'quản trị', 'backend', 'system manager', 'quản lí hệ thống',
                               'system engineer', 'infrastructure engineer', 'windows admin', 'linux admin', 'system analyst', 
                               'server administrator', 'cloud administrator', 'it administrator'],
    
    'Nhà phát triển web': ['web developer', 'phát triển web', 'thiết kế web', 'frontend', 'fullstack', 'web designer', 
                           'frontend engineer', 'fullstack engineer', 'wordpress developer', 'html developer', 'css developer', 
                           'javascript developer', 'web engineer', 'ui developer', 'web programmer', 'front-end', 'back-end', 
                           'react developer', 'angular developer'],
    
    'Nhà thiết kế đồ họa': ['graphic designer', 'thiết kế đồ họa', 'graphic', 'artist', 'multimedia designer', 'graphics artist', 
                            'visual designer', 'photoshop expert', 'illustrator', 'creative designer', 'branding designer', 
                            'ui artist', 'motion designer', '2d artist', '3d designer', 'digital artist', 'poster designer'],
    
    'Nhà khoa học dữ liệu': ['data scientist', 'khoa học dữ liệu', 'machine learning scientist', 'data researcher', 
                             'ml scientist', 'ai scientist', 'data science specialist', 'deep learning scientist', 
                             'data exploration', 'data analytics scientist'],
    
    'Nhà phân tích dữ liệu': ['data analyst', 'phân tích dữ liệu', 'business analyst', 'analyst', 'data analytics', 
                              'reporting analyst', 'data specialist', 'bi analyst', 'sql analyst', 'excel analyst', 
                              'business intelligence', 'data visualization', 'dashboard specialist', 'power bi developer'],
    
    'Nhà phát triển phần mềm': ['software developer', 'phát triển phần mềm', 'app developer', 'java developer', 'fullstack', 
                                'software architect', 'mobile application developer', 'desktop application developer', 
                                'application engineer', 'software creator', 'api developer', 'software solutions'],
    
    'Nhà thiết kế giao diện người dùng': ['ui designer', 'ux designer', 'giao diện', 'user interface', 'ui/ux', 'ux/ui', 
                                         'user experience', 'interaction designer', 'ui developer', 'ux researcher', 
                                         'product designer', 'ux writer', 'wireframe designer', 'prototype designer'],
    
    'Chuyên viên tư vấn CNTT': ['it consultant', 'tư vấn', 'brse', 'solution consultant', 'giải pháp', 'it advisory', 
                               'technical consultant', 'system consultant', 'it solutions expert', 'business it consultant'],
    
    'Chuyên viên hỗ trợ kỹ thuật': ['technical support', 'hỗ trợ kỹ thuật', 'it support', 'helpdesk', 'nhân viên kỹ thuật', 
                                    'camera', 'bảo hành', 'tech support', 'desktop support', 'it technician', 'support engineer', 
                                    'pc repair', 'network support', 'hardware technician'],
    
    'Chuyên viên bảo mật thông tin': ['information security', 'bảo mật', 'cybersecurity', 'an ninh', 'security engineer', 
                                     'it security', 'network security', 'security analyst', 'pentester', 'security operations', 
                                     'threat analyst', 'malware analyst'],
    
    'Kỹ sư mạng': ['network engineer', 'kỹ sư mạng', 'networking', 'network admin', 'network architect', 'wireless engineer', 
                   'network technician', 'ccna', 'network administrator', 'wan engineer', 'lan specialist', 'network infrastructure'],
    
    'Kỹ sư học máy': ['machine learning engineer', 'kỹ sư học máy', 'ml engineer', 'machine learning dev', 'ai developer', 
                      'ml architect', 'ai specialist', 'machine learning scientist', 'deep learning engineer', 'model developer'],
    
    'Kỹ sư đám mây': ['cloud engineer', 'kỹ sư đám mây', 'cloud architect', 'aws engineer', 'azure engineer', 'gcp engineer', 
                      'cloud admin', 'cloud solutions architect', 'cloud specialist', 'cloud devops'],
    
    'Kỹ sư phần mềm': ['software engineer', 'kỹ sư phần mềm', 'kỹ thuật phần mềm', 'swe', 'application engineer', 
                       'software dev', 'backend engineer', 'frontend engineer', 'software solutions engineer'],
    
    'Kỹ sư blockchain': ['blockchain engineer', 'kỹ sư blockchain', 'blockchain developer', 'smart contract developer', 
                         'crypto engineer', 'dapp developer', 'web3 developer', 'blockchain architect'],
    
    'Kỹ sư cơ sở dữ liệu': ['database engineer', 'cơ sở dữ liệu', 'database admin', 'dba', 'sql engineer', 'mysql admin', 
                            'oracle dba', 'postgresql engineer', 'database architect', 'data warehouse specialist'],
    
    'Kỹ sư trí tuệ nhân tạo': ['ai engineer', 'kỹ sư trí tuệ nhân tạo', 'artificial intelligence', 'ai architect', 
                               'ai developer', 'machine intelligence engineer', 'artificial intelligence scientist'],
    
    'Kỹ sư kiểm thử phần mềm': ['software tester', 'kiểm thử', 'test engineer', 'qa tester', 'quality assurance', 
                                'qa engineer', 'automation tester', 'manual tester', 'test analyst', 'sdet', 'software qa'],

    'Công nghệ thông tin (tổng quát)': ['it', 'công nghệ thông tin', 'cntt', 'information technology', 'it specialist', 
                                       'it technician', 'it officer', 'it expert', 'it solutions', 'it professional'],
}

# Hàm phân loại
def analyze_job_name(job: str):
    job = job.lower().strip()  # Chuyển về chữ thường và loại bỏ khoảng trắng
    for category, keywords in job_keywords.items():
        match = process.extractOne(job, keywords, scorer=fuzz.partial_ratio)
        if match and match[1] > 80:  # Ngưỡng độ tương đồng > 80%
            return category
    return job  # Trường hợp không khớp với bất kỳ công việc nào

## Ngày đăng tuyển

In [5]:
def analyze_posted_date(time: str):
    # Làm sạch dữ liệu đầu vào
    time = time.replace('một', '1')
    words = time.split()
    # Tìm vị trí của từ bắt đầu bằng số
    for i, word in enumerate(words):
        if word.isdigit():
            break
    if not words:
        return None  # Chuỗi không hợp lệ
    
    time_value = int(words[i])
    time_unit = words[i+1]

    # Lấy thời điểm hiện tại
    now = datetime.datetime.now()

    # Tính toán thời điểm tương ứng
    if time_unit == 'giây':
        result = now - datetime.timedelta(seconds=time_value)
    elif time_unit == 'phút':
        result = now - datetime.timedelta(minutes=time_value)
    elif time_unit == 'giờ':
        result = now - datetime.timedelta(hours=time_value)
    elif time_unit == 'ngày':
        result = now - datetime.timedelta(days=time_value)
    elif time_unit == 'tháng':
        result = now.replace(month=now.month - time_value)
    elif time_unit == 'năm':
        result = now.replace(year=now.year - time_value)
    else:
        return None

    return result

## Lương

In [6]:
def analyze_salary(salary: str):
    salary = salary.lower().strip()
    
    if salary == 'thương lượng':
        return None, None, None  # Thương lượng: cả 3 cột để trống
    
    elif salary == 'cạnh tranh':
        return 0, 0, 0  # Cạnh tranh: cả 3 cột bằng 0
    
    elif 'triệu' in salary:
        if 'trên' in salary:
            # Định dạng 'Trên X triệu'
            min_salary = int(float(salary.split('trên')[1].strip().replace('triệu', '').strip()) * 1_000_000)
            return min_salary, None, None
        elif '-' in salary:
            # Định dạng 'X triệu - Y triệu'
            min_salary, max_salary = map(
                lambda x: int(float(x.strip().replace('triệu', '').replace(',', '.')) * 1_000_000),
                salary.split('-')
            )
            avg_salary = (min_salary + max_salary) // 2
            return min_salary, max_salary, avg_salary
        else:
            # Định dạng 'X triệu'
            avg_salary = int(float(salary.replace('triệu', '').replace(',', '.').strip()) * 1_000_000)
            return None, None, avg_salary
        
    elif 'usd' in salary:
        if 'trên' in salary:
            # Định dạng 'Trên X triệu'
            min_salary = int(math.floor(float(salary.split('trên')[1].strip().replace('usd', '').replace(',', '').strip()) * 25_000 / 1_000_000) * 1_000_000)
            return min_salary, None, None
        elif '-' in salary:
            # Định dạng 'X USD - Y USD', chuyển sang VNĐ
            min_salary, max_salary = map(
                lambda x: int(math.floor(float(x.strip().replace('usd', '').replace(',', '').strip()) * 25_000 / 1_000_000) * 1_000_000),
                salary.split('-')
            )
            avg_salary = (min_salary + max_salary) // 2
            return min_salary, max_salary, avg_salary
        else:
            # Định dạng 'X USD'
            avg_salary = int(math.floor(float(salary.replace('usd', '').replace(',', '').strip()) * 25_000 / 1_000_000) * 1_000_000)
            return None, None, avg_salary
    
    else:
        return None, None, None  # Nếu không khớp định dạng, để trống

## Yêu cầu bằng cấp

In [7]:
def degree_assessment(degree: str):
    degree = degree.lower().strip()  # Chuyển về chữ thường và loại bỏ khoảng trắng đầu/cuối
    if 'đại học' in degree or 'cao đẳng' in degree or 'cử nhân' in degree:
        return 'Cử nhân'
    elif 'trung học' in degree:
        return 'Trung học'
    elif 'trung cấp' in degree:
        return 'Trung cấp'
    else:
        return degree

## Yêu cầu kinh nghiệm

In [8]:
def level_assessment(level: str):
    # Kiểm tra 'Chưa có kinh nghiệm'
    if 'Chưa có kinh nghiệm' in level:
        return 'Fresher'

    # Kiểm tra 'Dưới X năm'
    elif 'Dưới' in level:
        return 'Fresher'

    # Kiểm tra 'Trên X năm'
    elif 'Trên' in level or 'Hơn' in level:
        return 'Senior'

    elif '-' in level:
        # Định dạng 'X - Y năm kinh nghiệm'
        match = re.match(r'(\d+)\s*-\s*(\d+)\s*năm\s*', level)
        x = int(match.group(1))
        y = int(match.group(2))
        avg = (x + y) / 2
        
    else:
        # Định dạng 'X năm'
        match = re.match(r'(\d+)\s*năm\s*', level)
        if match:
            avg = int(match.group(1))

    # Đánh giá cấp độ
    if avg <= 1:
        return 'Fresher'
    elif 1 < avg < 3:
        return 'Junior'
    elif 3 <= avg < 5:
        return 'Middle'
    else:
        return 'Senior'

# Truy cập và trích xuất dữ liệu từ trang web

## CareerLink

In [9]:
careerlink_url = 'https://www.careerlink.vn/viec-lam/cntt-phan-mem/19'
max_pages = 5  # Số trang tối đa muốn lấy (thay đổi tùy ý)

try:
    for page in range(1, max_pages + 1):
        # Mở trang web CareerLink
        url = f'{careerlink_url}?page={page}' if page > 1 else careerlink_url
        driver.get(url)
        
        # Chờ CAPTCHA xuất hiện nếu có
        try:
            captcha_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'g-recaptcha')))
            print("Hãy giải CAPTCHA thủ công trên trình duyệt và nhấn Enter để tiếp tục...")
            input()  # Dừng chương trình để bạn giải CAPTCHA thủ công
        except:
            pass
            
        # Trích xuất các công việc
        jobs = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'list-group-item')))

        for job in jobs:
            try:
                job_text = job.text.split('\n')  # Tách thông tin thành từng dòng
                if len(job_text) < 5:  # Bỏ qua các mục không hợp lệ
                    continue
                index_save = next((i for i, line in enumerate(job_text) if 'Lưu' in line), None)  # Bỏ các dòng 'Lưu' và trên nó
                if index_save is not None:
                    job_text = job_text[index_save + 1:]

                # Click vào công việc để chuyển đến trang thông tin chi tiết của công việc đó
                job_detail_link = job.find_element(By.CSS_SELECTOR, 'a.job-link.clickable-outside')
                job_detail_link.click()

                # Chuyển sang tab mới
                window_handles = driver.window_handles
                driver.switch_to.window(window_handles[-1])
                
                # Chờ CAPTCHA xuất hiện nếu có
                try:
                    captcha_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'g-recaptcha')))
                    print("Hãy giải CAPTCHA thủ công trên trình duyệt và nhấn Enter để tiếp tục...")
                    input()  # Dừng chương trình để bạn giải CAPTCHA thủ công
                except:
                    pass

                # Tìm các thông tin chi tiết
                job_details = WebDriverWait(driver, 3).until(
                    EC.visibility_of_all_elements_located((By.XPATH, "//div[@id='section-job-summary']/*"))
                )
                job_detail_text = [item for job_detail in job_details for item in job_detail.text.split('\n')]
                
                # Tổng hợp các thông tin thu thập được
                website_name = 'CareerLink'
                job_name = analyze_job_name(job_text[0])
                company_name = job_text[1]
                place = job_text[2]
                posted_date = analyze_posted_date(job_text[3])
                salary = analyze_salary(job_text[4])
                position = job_text[6]
                if 'Học vấn' in job_detail_text:
                    degree_required = degree_assessment(job_detail_text[job_detail_text.index('Học vấn') + 1])
                else:
                    degree_required = None
                experience_required = level_assessment(job_detail_text[job_detail_text.index('Kinh nghiệm') + 1])

                # Thêm dữ liệu vào DataFrame
                new_row = {
                    'Trang thu thập': 'CareerLink',
                    'Tên công ty': company_name,
                    'Tên công việc': job_name,
                    'Vị trí ứng tuyển': position,
                    'Yêu cầu bằng cấp': degree_required,
                    'Yêu cầu kinh nghiệm': experience_required,
                    'Địa điểm': place,
                    'Ngày đăng tuyển': posted_date,
                    'Lương tối thiểu': salary[0],
                    'Lương tối đa': salary[1],
                    'Lương TB': salary[2]
                }
                df.loc[len(df)] = new_row
            
            except Exception as e:
                print(f'Lỗi khi xử lý công việc: {e}')
                
            finally:
                # Trở lại trang chủ
                if len(window_handles) > 1:  # Đảm bảo có ít nhất 2 window (window chính + 1 window mới)
                    driver.close()
                    driver.switch_to.window(window_handles[0])
            
except Exception as e:
    print(f'Có lỗi xảy ra trong quá trình thu thập dữ liệu: {e}')

Hãy giải CAPTCHA thủ công trên trình duyệt và nhấn Enter để tiếp tục...


## việclàm24h

In [10]:
vieclam24h_url_1 = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids%5B%5D=8&occupation_ids%5B%5D=7&'
vieclam24h_url_2 = '&sort_q=priority_max%2Cdesc'

try:
    for page in range(1, 5 + 1):
        # Mở trang web việclàm24h
        url = f'{vieclam24h_url_1}page={page}{vieclam24h_url_2}'
        driver.get(url)
        
        # Chờ CAPTCHA xuất hiện nếu có
        try:
            captcha_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'g-recaptcha')))
            print("Hãy giải CAPTCHA thủ công trên trình duyệt và nhấn Enter để tiếp tục...")
            input()  # Dừng chương trình để bạn giải CAPTCHA thủ công
        except:
            pass

        # Trích xuất các công việc
        jobs = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'flex.flex-col.gap-3')))[0].find_elements(By.XPATH, './*')
        
        for job in jobs:
            try:
                job_text = job.text.split('\n')  # Tách thông tin thành từng dòng

                # Click vào công việc để chuyển đến trang thông tin chi tiết của công việc đó
                job.click()
                
                # Chuyển sang tab mới
                window_handles = driver.window_handles
                driver.switch_to.window(window_handles[-1])
                
                # Chờ CAPTCHA xuất hiện nếu có
                try:
                    captcha_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'g-recaptcha')))
                    print("Hãy giải CAPTCHA thủ công trên trình duyệt và nhấn Enter để tiếp tục...")
                    input()  # Dừng chương trình để bạn giải CAPTCHA thủ công
                except:
                    pass

                # Tìm các thông tin chi tiết
                job_details = WebDriverWait(driver, 3).until(
                    EC.visibility_of_all_elements_located((By.XPATH, "//div[@id='__next']/div/main/div/div[3]/div/div[2]/div/div/div/h3"))
                )
                job_detail_text = [item for job_detail in job_details for item in job_detail.text.split('\n')]
                
                # Tổng hợp các thông tin thu thập được
                website_name = 'vieclam24h'
                job_name = analyze_job_name(job_text[0])
                company_name = job_text[1]
                salary = analyze_salary(job_text[2])
                place = job_text[3]
                posted_date = datetime.datetime.strptime(job_detail_text[job_detail_text.index('Ngày đăng') + 1], '%d/%m/%Y')
                position = job_detail_text[job_detail_text.index('Cấp bậc') + 1]
                if 'Yêu cầu bằng cấp' in job_detail_text:
                    degree_required = degree_assessment(job_detail_text[job_detail_text.index('Yêu cầu bằng cấp') + 1])
                else:
                    degree_required = None
                experience_required = level_assessment(job_detail_text[job_detail_text.index('Yêu cầu kinh nghiệm') + 1])
                
                # Thêm dữ liệu vào DataFrame
                new_row = {
                    'Trang thu thập': 'việclàm24h',
                    'Tên công ty': company_name,
                    'Tên công việc': job_name,
                    'Vị trí ứng tuyển': position,
                    'Yêu cầu bằng cấp': degree_required,
                    'Yêu cầu kinh nghiệm': experience_required,
                    'Địa điểm': place,
                    'Ngày đăng tuyển': posted_date,
                    'Lương tối thiểu': salary[0],
                    'Lương tối đa': salary[1],
                    'Lương TB': salary[2]
                }
                df.loc[len(df)] = new_row

            except Exception as e:
                print(f'Lỗi khi xử lý công việc: {e}')
                
            finally:
                # Trở lại trang chủ
                if len(window_handles) > 1:  # Đảm bảo có ít nhất 2 window (window chính + 1 window mới)
                    driver.close()
                    driver.switch_to.window(window_handles[0])
        
except Exception as e:
    print(f'Có lỗi xảy ra trong quá trình thu thập dữ liệu: {e}')

# Lưu DataFrame vào file csv

In [11]:
# In da DataFrame
df.to_csv('job_data.csv', index=True, index_label='STT', encoding='utf-8-sig')
df

,Trang thu thập,Tên công ty,Tên công việc,Vị trí ứng tuyển,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Địa điểm,Ngày đăng tuyển,Lương tối thiểu,Lương tối đa,Lương TB
0,CareerLink,CÔNG TY CỔ PHẦN KINH DOANH F88,middle tester (có kinh nghiệm tài chính - ngân...,Nhân viên,Cử nhân,Middle,Hà Nội,2024-12-24 09:55:45.963036,None,None,None
1,CareerLink,TRƯỜNG ĐẠI HỌC KINH TẾ TÀI CHÍNH TP.HCM,Kỹ sư phần mềm,Nhân viên,thạc sĩ,Middle,Hồ Chí Minh,2024-12-23 15:55:53.568545,None,None,None
2,CareerLink,Công Ty Cổ Phần Bán Lẻ Kỹ Thuật Số FPT,Chuyên viên hỗ trợ kỹ thuật,Nhân viên,Trung học,Fresher,Gia Lai,2024-12-23 13:56:13.085642,6000000,11000000,8500000
3,CareerLink,TRƯỜNG ĐẠI HỌC KINH TẾ TÀI CHÍNH TP.HCM,giảng viên ngành công nghệ tài chính,Nhân viên,thạc sĩ,Middle,Hồ Chí Minh,2024-12-23 14:56:19.548093,None,None,None
4,CareerLink,CÔNG TY CỔ PHẦN TRUYỀN THÔNG VÀNG CHÂU Á,Chuyên viên hỗ trợ kỹ thuật,Nhân viên,Cử nhân,Junior,Hồ Chí Minh,2024-12-23 13:56:31.059003,12000000,15000000,13500000
...,...,...,...,...,...,...,...,...,...,...,...
395,việclàm24h,Công Ty Cổ Phần Tập Đoàn Techhub,Nhà thiết kế đồ họa,Cộng tác viên,None,Fresher,Hà Nội,2024-12-13 00:00:00.000000,3000000,10000000,6500000
396,việclàm24h,Công Ty Cổ Phần A.R.T,Chuyên viên hỗ trợ kỹ thuật,Chuyên viên- nhân viên,Cử nhân,Fresher,Hà Nội,2024-12-13 00:00:00.000000,7000000,11000000,9000000
397,việclàm24h,Thanh Mai Digital,kỹ thuật viên laptop và máy ảnh,Chuyên viên- nhân viên,None,Fresher,TP.HCM,2024-12-13 00:00:00.000000,7000000,20000000,13500000
398,việclàm24h,Công Ty CP Chứng Khoán Fpt,Kỹ sư cơ sở dữ liệu,Chuyên viên- nhân viên,None,Fresher,Hà Nội,2024-12-13 00:00:00.000000,10000000,20000000,15000000


Đóng trình duyệt:

In [12]:
driver.quit()

# Tiền xử lý dữ liệu


In [27]:
# import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# Đọc dữ liệu từ tệp CSV
df = pd.read_csv('job_data.csv')
df.head()

,STT,Trang thu thập,Tên công ty,Tên công việc,Vị trí ứng tuyển,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Địa điểm,Ngày đăng tuyển,Lương tối thiểu,Lương tối đa,Lương TB
0,0,CareerLink,CÔNG TY TNHH DAIWA VIỆT NAM,Chuyên viên hỗ trợ kỹ thuật,Nhân viên,Cử nhân,Fresher,Đà Nẵng,2024-12-19 15:24:28.662632,7000000.0,10000000.0,8500000.0
1,1,CareerLink,FPT Telecom,Nhà phân tích dữ liệu,Sinh viên / Thực tập sinh,Cử nhân,Junior,Hồ Chí Minh,2024-12-19 15:24:31.499363,7000000.0,10000000.0,8500000.0
2,2,CareerLink,Công Ty Cổ Phần Tập Đoàn Omi (OmiGroup),Lập trình viên,Nhân viên,Cử nhân,Junior,Hà Nội,2024-12-19 14:24:34.453881,NaN,NaN,NaN
3,3,CareerLink,CÔNG TY CỔ PHẦN SYNOPEX VIỆT NAM,Lập trình viên,Nhân viên,Cử nhân,Junior,Bắc Ninh,2024-12-19 11:24:37.033472,NaN,NaN,NaN
4,4,CareerLink,Công Ty Cổ Phần Tập Đoàn Omi (OmiGroup),Lập trình viên,Nhân viên,Cử nhân,Senior,Hà Nội,2024-12-19 15:24:39.522462,NaN,NaN,NaN


Xử lý dữ liệu bị thiếu

In [29]:
df.isnull().sum()

STT                      0
Trang thu thập           0
Tên công ty              0
Tên công việc            0
Vị trí ứng tuyển         0
Yêu cầu bằng cấp        49
Yêu cầu kinh nghiệm      0
Địa điểm                 0
Ngày đăng tuyển          0
Lương tối thiểu        115
Lương tối đa           125
Lương TB               121
dtype: int64

In [30]:
# Convert 'Ngày đăng tuyển' to datetime
df['Ngày đăng tuyển'] = pd.to_datetime(df['Ngày đăng tuyển'], errors='coerce')

# Fill missing values for 'Yêu cầu bằng cấp' with 'Không yêu cầu'
df['Yêu cầu bằng cấp'].fillna('Không yêu cầu', inplace=True)

# Fill missing values for salary columns with 0
df['Lương tối thiểu'].fillna(0, inplace=True)
df['Lương tối đa'].fillna(0, inplace=True)
df['Lương TB'].fillna(0, inplace=True)

# Verify the changes
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   STT                  334 non-null    int64         
 1   Trang thu thập       334 non-null    object        
 2   Tên công ty          334 non-null    object        
 3   Tên công việc        334 non-null    object        
 4   Vị trí ứng tuyển     334 non-null    object        
 5   Yêu cầu bằng cấp     334 non-null    object        
 6   Yêu cầu kinh nghiệm  334 non-null    object        
 7   Địa điểm             334 non-null    object        
 8   Ngày đăng tuyển      334 non-null    datetime64[ns]
 9   Lương tối thiểu      334 non-null    float64       
 10  Lương tối đa         334 non-null    float64       
 11  Lương TB             334 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(7)
memory usage: 31.4+ KB


C:\Users\Dell\AppData\Local\Temp\ipykernel_18120\61948994.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Yêu cầu bằng cấp'].fillna('Không yêu cầu', inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_18120\61948994.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

,STT,Trang thu thập,Tên công ty,Tên công việc,Vị trí ứng tuyển,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Địa điểm,Ngày đăng tuyển,Lương tối thiểu,Lương tối đa,Lương TB
0,0,CareerLink,CÔNG TY TNHH DAIWA VIỆT NAM,Chuyên viên hỗ trợ kỹ thuật,Nhân viên,Cử nhân,Fresher,Đà Nẵng,2024-12-19 15:24:28.662632,7000000.0,10000000.0,8500000.0
1,1,CareerLink,FPT Telecom,Nhà phân tích dữ liệu,Sinh viên / Thực tập sinh,Cử nhân,Junior,Hồ Chí Minh,2024-12-19 15:24:31.499363,7000000.0,10000000.0,8500000.0
2,2,CareerLink,Công Ty Cổ Phần Tập Đoàn Omi (OmiGroup),Lập trình viên,Nhân viên,Cử nhân,Junior,Hà Nội,2024-12-19 14:24:34.453881,0.0,0.0,0.0
3,3,CareerLink,CÔNG TY CỔ PHẦN SYNOPEX VIỆT NAM,Lập trình viên,Nhân viên,Cử nhân,Junior,Bắc Ninh,2024-12-19 11:24:37.033472,0.0,0.0,0.0
4,4,CareerLink,Công Ty Cổ Phần Tập Đoàn Omi (OmiGroup),Lập trình viên,Nhân viên,Cử nhân,Senior,Hà Nội,2024-12-19 15:24:39.522462,0.0,0.0,0.0


Nhận thấy mục lương thiếu khá nhiều và là yếu tố rất quan trọng vì thế nên nhóm tiến hành xử lý điền thông tin bởi những yếu tố liên quan


Lấy ra những dòng đủ giá trị lương trong data để tiến hành phân tích các yếu tố liên quan

In [31]:
# Loại bỏ giá trị lương bị trống hoặc bằng 0
df_filtered = df[(df['Lương tối thiểu'] > 0) & (df['Lương tối đa'] > 0)]

In [17]:
df_filtered.head()

,STT,Trang thu thập,Tên công ty,Tên công việc,Vị trí ứng tuyển,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Địa điểm,Ngày đăng tuyển,Lương tối thiểu,Lương tối đa,Lương TB
0,0,CareerLink,CÔNG TY TNHH DAIWA VIỆT NAM,Chuyên viên hỗ trợ kỹ thuật,Nhân viên,Cử nhân,Fresher,Đà Nẵng,2024-12-19 15:24:28.662632,7000000.0,10000000.0,8500000.0
1,1,CareerLink,FPT Telecom,Nhà phân tích dữ liệu,Sinh viên / Thực tập sinh,Cử nhân,Junior,Hồ Chí Minh,2024-12-19 15:24:31.499363,7000000.0,10000000.0,8500000.0
7,7,CareerLink,PILGRIMAGE VILLAGE HUẾ BOUTIQUE RESORT & SPA,Công nghệ thông tin (tổng quát),Trưởng nhóm / Giám sát,Cử nhân,Middle,Thừa Thiên - Huế,2024-12-19 10:24:47.830684,10000000.0,15000000.0,12500000.0
10,10,CareerLink,PILGRIMAGE VILLAGE HUẾ BOUTIQUE RESORT & SPA,Công nghệ thông tin (tổng quát),Trưởng nhóm / Giám sát,Cử nhân,Middle,Thừa Thiên - Huế,2024-12-19 10:24:56.195206,10000000.0,15000000.0,12500000.0
21,21,CareerLink,CÔNG TY TNHH QUICK VIỆT NAM,Lập trình viên,Nhân viên,Cử nhân,Middle,Hồ Chí Minh,2024-12-19 06:25:26.339503,15000000.0,25000000.0,20000000.0


## Xác định các yếu tố ảnh hưởng
Xem xét các cột dữ liệu để xác định các yếu tố có khả năng tác động đến lương:

- Vị trí ứng tuyển: Một số vị trí như lập trình viên, scrum master thường có mức lương cao hơn.
- Kinh nghiệm (Junior, Middle, Fresher): Cấp bậc và kinh nghiệm thường tác động mạnh đến lương.
- Trình độ học vấn: Các vai trò yêu cầu trình độ học vấn cao thường trả lương tốt hơn.
- Địa điểm: Mức lương ở Hà Nội, Hồ Chí Minh thường cao hơn ở các tỉnh nhỏ.

In [24]:
# df_filtered['Vị trí ứng tuyển'].unique()
# df_filtered['Yêu cầu bằng cấp'].unique()
# df_filtered['Yêu cầu kinh nghiệm'].unique()
df_filtered['Địa điểm'].unique()

array(['Đà Nẵng', 'Hồ Chí Minh', 'Thừa Thiên - Huế', 'Ninh Thuận',
       'Tiền Giang', 'Gia Lai', 'Đồng Tháp', 'Quảng Nam', 'Bắc Giang',
       'Hà Nội', 'Bình Dương', 'Quảng Ngãi', 'Lâm Đồng', 'Hậu Giang',
       'Bắc Ninh', 'Hải Phòng', 'Hồ Chí Minh Hà Nội', 'Quảng Bình',
       'Khánh Hòa', 'Bến Tre', 'Đắk Lắk', 'Nhật Bản', 'Kiên Giang',
       'Bình Định', 'Đăk Nông', 'An Giang', 'Bạc Liêu', 'Hải Dương',
       'TP.HCM', 'Đồng Nai', 'TP.HCM, Bình Dương', 'Phú Thọ', 'Long An',
       'Nước ngoài', 'Tây Ninh', 'TP.HCM, Đà Nẵng, Thừa Thiên Huế, +2',
       'Kiên Giang, An Giang, TP.HCM, +2', 'Vĩnh Phúc, Hà Nội',
       'Đà Nẵng, Hà Nội, TP.HCM', 'Hà Nội, TP.HCM', 'Bình Thuận',
       'Hà Nội, TP.HCM, Đà Nẵng'], dtype=object)

Xử lý dữ liệu trùng lặp

In [32]:
df.duplicated().sum()

0

In [33]:
df = df.drop_duplicates()
df.shape

(334, 12)

In [35]:
# Tính median theo tổ hợp nhóm các yếu tố
median_by_groups = df_filtered.groupby(
    ['Vị trí ứng tuyển', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Địa điểm']
)['Lương tối thiểu'].median().reset_index()

# Chuyển đổi thành từ điển để ánh xạ nhanh
group_median_dict = median_by_groups.set_index(
    ['Vị trí ứng tuyển', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Địa điểm']
)['Lương tối thiểu'].to_dict()

# Hàm điền giá trị lương tối thiểu
def group_based_imputation(row):
    if pd.notna(row['Lương tối thiểu']) and row['Lương tối thiểu'] > 0:
        return row['Lương tối thiểu']  # Giữ nguyên nếu lương tối thiểu đã hợp lệ

    # Ánh xạ tổ hợp các yếu tố để điền lương tối thiểu
    key = (row['Vị trí ứng tuyển'], row['Yêu cầu bằng cấp'], row['Yêu cầu kinh nghiệm'], row['Địa điểm'])
    if key in group_median_dict:
        return group_median_dict[key]

    # Nếu không có trong nhóm, điền bằng median toàn cục
    return df_filtered['Lương tối thiểu'].median()

# Áp dụng hàm
df['Lương tối thiểu'] = df.apply(group_based_imputation, axis=1)

# Kiểm tra số lượng lương còn thiếu
print("Số lượng giá trị lương còn thiếu:", df['Lương tối thiểu'].isna().sum())

Số lượng giá trị lương còn thiếu: 0


In [38]:
df

,STT,Trang thu thập,Tên công ty,Tên công việc,Vị trí ứng tuyển,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Địa điểm,Ngày đăng tuyển,Lương tối thiểu,Lương tối đa,Lương TB
0,0,CareerLink,CÔNG TY TNHH DAIWA VIỆT NAM,Chuyên viên hỗ trợ kỹ thuật,Nhân viên,Cử nhân,Fresher,Đà Nẵng,2024-12-19 15:24:28.662632,7000000.0,10000000.0,8500000.0
1,1,CareerLink,FPT Telecom,Nhà phân tích dữ liệu,Sinh viên / Thực tập sinh,Cử nhân,Junior,Hồ Chí Minh,2024-12-19 15:24:31.499363,7000000.0,10000000.0,8500000.0
2,2,CareerLink,Công Ty Cổ Phần Tập Đoàn Omi (OmiGroup),Lập trình viên,Nhân viên,Cử nhân,Junior,Hà Nội,2024-12-19 14:24:34.453881,13000000.0,19000000.0,0.0
3,3,CareerLink,CÔNG TY CỔ PHẦN SYNOPEX VIỆT NAM,Lập trình viên,Nhân viên,Cử nhân,Junior,Bắc Ninh,2024-12-19 11:24:37.033472,10000000.0,15000000.0,0.0
4,4,CareerLink,Công Ty Cổ Phần Tập Đoàn Omi (OmiGroup),Lập trình viên,Nhân viên,Cử nhân,Senior,Hà Nội,2024-12-19 15:24:39.522462,10000000.0,15000000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
329,329,CareerLink,Công Ty TNHH Gia Hòa,Công nghệ thông tin (tổng quát),Chuyên viên- nhân viên,Cử nhân,Middle,Bình Thuận,2024-12-18 00:00:00.000000,12000000.0,30000000.0,21000000.0
330,330,CareerLink,Công Ty TNHH Ggame,Kinh doanh,Chuyên viên- nhân viên,Không yêu cầu,Junior,TP.HCM,2024-12-18 00:00:00.000000,10000000.0,20000000.0,15000000.0
331,331,CareerLink,Công Ty Tnhh Eup Solution,Kinh doanh,Chuyên viên- nhân viên,Không yêu cầu,Junior,Hà Nội,2024-12-18 00:00:00.000000,15000000.0,25000000.0,20000000.0
332,332,CareerLink,Công Ty Cổ Phần Kỹ Thuật Hoàng Thịnh,Kinh doanh,Chuyên viên- nhân viên,Cử nhân,Fresher,TP.HCM,2024-12-18 00:00:00.000000,10000000.0,18000000.0,14000000.0


In [39]:
# Tính median theo tổ hợp nhóm các yếu tố
median_by_groups = df_filtered.groupby(
    ['Vị trí ứng tuyển', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Địa điểm']
)['Lương tối đa'].median().reset_index()

# Chuyển đổi thành từ điển để ánh xạ nhanh
group_median_dict = median_by_groups.set_index(
    ['Vị trí ứng tuyển', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Địa điểm']
)['Lương tối đa'].to_dict()

# Hàm điền giá trị Lương tối đa
def group_based_imputation(row):
    if pd.notna(row['Lương tối đa']) and row['Lương tối đa'] > 0:
        return row['Lương tối đa']  # Giữ nguyên nếu Lương tối đa đã hợp lệ

    # Ánh xạ tổ hợp các yếu tố để điền Lương tối đa
    key = (row['Vị trí ứng tuyển'], row['Yêu cầu bằng cấp'], row['Yêu cầu kinh nghiệm'], row['Địa điểm'])
    if key in group_median_dict:
        return group_median_dict[key]

    # Nếu không có trong nhóm, điền bằng median toàn cục
    return df_filtered['Lương tối đa'].median()

# Áp dụng hàm
df['Lương tối đa'] = df.apply(group_based_imputation, axis=1)

# Kiểm tra số lượng lương còn thiếu
print("Số lượng giá trị lương còn thiếu:", df['Lương tối đa'].isna().sum())

Số lượng giá trị lương còn thiếu: 0
